# Conversation with Software Engineering Agent

SWE-agent(SoftWare Engineering Agent) is an agent designed for solving real world software engineering problems, such as fixing github issues.
More details can be found in the project's [homepage](https://swe-agent.com/) and related [github repo](https://swe-agent.com/).

In the example here, we partially implement the SWE-agent, and provide a simple example of how to use the implemented SWE-agent to fix a bug in a python file.
You should note that currently how to enable agents with stronger programming capabilities remains an open challenge, and the performance of the paritially implemented SWE-agent is not guaranteed.

## Prerequisites

- Follow [READMD.md](https://github.com/modelscope/agentscope) to install AgentScope. We require the lastest version, so you should build from source by running `pip install -e .` instead of intalling from pypi. 
- Prepare a model configuration. AgentScope supports both local deployed model services (CPU or GPU) and third-party services. More details and example model configurations please refer to our [tutorial](https://modelscope.github.io/agentscope/en/tutorial/203-model.html).
- Understand the ServiceToolkit module and how to use it to pre-process the tool functions for LLMs. You can refer to the [ReAct agent example](../conversation_with_react_agent/main.ipynb) and you should also refer to the [tutorial](https://modelscope.github.io/agentscope/en/tutorial/204-service.html) for service functions.


## Note

- The example is tested with the following models. For other models, you may need to adjust the prompt.
    - gpt-4
    - dashscope_chat (qwen-max)
    - gemini_chat (gemini-pro)
    - ollama_chat (llama3_8b)
- How to enable agents with stronger programming capabilities remains an open challenge, and the current implementations are not perfect. Please feel free to explore it yourself.

In [1]:
YOUR_MODEL_CONFIGURATION_NAME = "{YOUR_MODEL_CONFIGURATION_NAME}"

YOUR_MODEL_CONFIGURATION = {
    "model_type": "xxx", 
    "config_name": YOUR_MODEL_CONFIGURATION_NAME
    
    # ...
}

## Step 1: Initialize the AgentScope environment and SWE-agent

Here we init the agentscope environment and load the SWE-agent.

The code of SWE-agent is in `swe_agent.py`, and the related prompts are in `swe_agent_prompts.py`.

If you are interested in the details, please refer to the code and the origianl SWE-agent repo [here](https://github.com/princeton-nlp/SWE-agent).

In [ ]:
from swe_agent import SWEAgent

import agentscope

agentscope.init(
    model_configs=YOUR_MODEL_CONFIGURATION,
    project="Conversation with SWE-agent",
)

agent = SWEAgent(
    name="assistant",
    model_config_name=YOUR_MODEL_CONFIGURATION_NAME,
)


## Step 2: Create the code to be processed by the SWE-agent

Here we use the `write_file` function to write the following code into `gcd.py`.
The code here is a wrong implementation of the [Greatest Common Divisor (GCD) algorithm](https://en.wikipedia.org/wiki/Euclidean_algorithm).
We will ask the SWE-agent to correct it in our next step.

In [3]:
from swe_agent_service_func import write_file

# here the gcd code is written in a wrong way
wrong_gcd_text = """
def gcd(a, b):
    if a == 0:
        return b
    while a != 0:
        a, b = b, a
    return b

def lcm(a, b):
    return (a * b) // gcd(a, b)

# testing on GCD and LCM functions
print("GCD of 12 and 18 is:", gcd(12, 18))
print("LCM of 12 and 18 is:", lcm(12, 18))
"""

write_file("gcd.py", wrong_gcd_text)


{'status': <ServiceExecStatus.SUCCESS: 1>,
 'content': 'WRITE OPERATION:\nYou have written to "gcd.py"                 on these lines: 0:-1.\ndef gcd(a, b):\n    if a == 0:\n        return b\n    while a != 0:\n        a, b = b, a\n    return b\n\ndef lcm(a, b):\n    return (a * b) // gcd(a, b)\n\n# testing on GCD and LCM functions\nprint("GCD of 12 and 18 is:", gcd(12, 18))\nprint("LCM of 12 and 18 is:", lcm(12, 18))\n\n'}

## Step 3: Ask the SWE-agent to correct the code

Here we simple ask the SWE-agent to correct the code in `gcd.py`.

In [4]:
from loguru import logger
from agentscope.message import Msg

mss = Msg(
    name="user", 
    content="The file in gcd.py is not returning the correct answer, can you fix the file for me?", 
    role="user"
)
logger.chat(mss)
answer_mss = agent(mss)

user: The file in gcd.py is not returning the correct answer, can you fix the file for me?
assistant: {
    "thought": "First thing I'll do is read the content of gcd.py file to understand the existing logic.",
    "action": {
        "name": "read_file",
        "arguments": {
            "file_path": "gcd.py"
        }
    }
}
assistant: 
====Observation====

def gcd(a, b):
    if a == 0:
        return b
    while a != 0:
        a, b = b, a
    return b

def lcm(a, b):
    return (a * b) // gcd(a, b)

# testing on GCD and LCM functions
print("GCD of 12 and 18 is:", gcd(12, 18))
print("LCM of 12 and 18 is:", lcm(12, 18))


assistant: {
    "thought": "I can see the error in the gcd calculation. The gcd calculation in the file is incorrect. The correct way to find the gcd of two numbers using the Euclidean algorithm is to continually reduce a and b by finding the remainder of a % b until b is 0, at which point a will be the gcd of the original a and b values. In the given code, inste

### Conlusion

After a few iterations, the SWE-agent assistant finish the job successfully, and the code is now working fine.

Above we shown a example of how to use the SWE-agent to fix code errors.
Although the design of the SWE-agent is primarily aimed at addressing GitHub issues, with modifications, it can also be utilized for more general programming tasks.

Currently, how to enable agent with general programming ablities remains a challenging open question, with the efficacy of agent programming potentially influenced by factors such as prompt construction, model capabilities, and the complexity of the task at hand. Here we just provide an interesting toy example. 

We encourage users to experiment by altering the prompts within this example or by assigning different tasks to the agent, among other methods of exploration. Please feel free to experiment and explore on your own. The AgentScope team will continue to provide updates, enhancing the capabilities of the Programming Agents in the future!